# Stable Baselines3 Tutorial - Getting Started
- Create a basic RL model.
- Train it in a custom env.
- Evaluate it.

Also:
- Record a video of the agent.

## Definition of model and env

In [1]:
# Previously, install ffmpeg freeglut3-dev xvfb  (for visualization).

# Imports
import gym
import numpy as np

# Import the RL model.
from stable_baselines3 import PPO # RL algorithm.
from stable_baselines3.ppo.policies import MlpPolicy # MLP as policy for a feature vector (no image).

In [2]:
# Create the Gym env:
# The action space is deduced from the env. action space.
env = gym.make('CartPole-v1')

# Instantiate the agent:
# PPO (actor-critic, using a value function to improve the policy).
# MlpPolicy because the observation of the CartPole task is a feature vector, not images.
model = PPO(MlpPolicy, env, verbose=0)

## Training and evaluation

In [3]:
# Helper function to evaluate the agent.
def evaluate(model, num_episodes=100):
    env = model.get_env() # Use the env. of the model.
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset() # Initialize env; first observation.
        while not done:
            action, _states = model.predict(obs) # states are only useful with LSTM policies.
            obs, reward, done, info = env.step(action) # returns arrays (bc vecotirzed env.)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
    
    mean_episode_reward = np.mean(all_episode_rewards)
    print(f'Mean reward: {mean_episode_reward}, Num episodes: {num_episodes}')

    return mean_episode_reward

In [ ]:
# Random agent, before training
mean_reward_before_train = evaluate(model, num_episodes=100)

Mean reward: 22.219999313354492, Num episodes: 100


In [ ]:
# Using Stable Baselines 3's helper.
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f'Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}')


Mean reward: 40.06 +/- 16.02


In [ ]:
# Train the agent adn evaluate it.
model.learn(total_timesteps=10000)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f'Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}')


/Users/Alberto/Code/RLtest1/.venv/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Mean reward: 396.80 +/- 114.51


## Video recording

In [ ]:
# Set up fake display; otherwise rendering will fail.
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.
_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2
_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local


In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
# Record and save the video.
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

Error: Session cannot generate requests